In [1]:
import sqlite3
import sqlalchemy
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import yfinance as yf
from datetime import date
from components.tickers import *

start_date = date.fromisoformat("2015-01-03")

In [3]:
df = pd.DataFrame()
tickerList = delisted.split()

for ticker in tickerList:
    df[ticker] = web.DataReader( ticker, 'stooq', start=start_date).Close

C:\Users\pauli\AppData\Local\Temp\ipykernel_14776\3427797853.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ticker] = web.DataReader( ticker, 'stooq', start=start_date).Close
C:\Users\pauli\AppData\Local\Temp\ipykernel_14776\3427797853.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ticker] = web.DataReader( ticker, 'stooq', start=start_date).Close
C:\Users\pauli\AppData\Local\Temp\ipykernel_14776\3427797853.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

AttributeError: 'DataFrame' object has no attribute 'Close'

In [50]:
engine = sqlalchemy.create_engine('sqlite:///database.db')

In [34]:
ticker1 = yf.Tickers(all_tickers)
dataframe1 = ticker1.history(
    start=start_date, interval="1d", rounding=True, auto_adjust=True).Close

[*********************100%***********************]  680 of 680 completed


In [51]:
dataframe1 = dataframe1.tz_localize(None)
df = df.tz_localize(None)
companiesFrame = pd.concat([dataframe1, df], axis=1)
companiesFrame = companiesFrame.reset_index()
companiesFrame['Date'] = pd.DatetimeIndex(companiesFrame['Date'])

In [52]:
companiesFrame.tail()

,Date,A,AAP,AAPL,ABB,ABM,ABR,ABT,ACGL,ACIW,...,PSA,PSTG,PSX,PVH,PWR,PXD,QDEL,QSR,QTWO,RAD
2031,2023-01-30,151.74,149.88,143.00,34.40,45.44,14.61,109.81,63.99,27.79,...,298.89,28.32,106.42,87.25,149.10,229.83,85.12,66.59,32.04,3.46
2032,2023-01-31,152.08,152.28,144.29,34.85,46.91,14.93,110.55,64.35,27.93,...,304.34,28.94,100.27,89.90,152.19,230.35,85.61,66.93,32.72,3.65
2033,2023-02-01,155.45,156.84,145.43,35.33,47.23,15.16,111.82,64.18,28.20,...,305.94,29.55,97.99,93.09,152.87,223.96,88.51,67.05,33.76,3.69
2034,2023-02-02,155.50,155.24,150.82,33.93,47.94,15.34,112.25,61.35,28.44,...,311.46,30.27,98.98,92.35,151.68,220.00,91.46,67.55,35.78,3.82
2035,2023-02-03,154.55,152.17,154.50,34.39,48.01,15.31,111.79,61.26,28.16,...,308.47,29.91,98.81,92.06,152.43,220.00,89.70,67.65,34.39,3.76


In [44]:
indexes = yf.Ticker('SPY')
indexes_data = indexes.history(
    start=start_date, interval="1d", rounding=True, auto_adjust=True).Close
indexes_data = indexes_data.tz_localize(None)
indexes_data = indexes_data.reset_index()
indexes_data["Date"] = pd.DatetimeIndex(indexes_data['Date'])

In [53]:
companiesFrame.to_sql("stockData", engine, if_exists='replace', index=False)
indexes_data.to_sql("indexes", engine, if_exists='replace', index=False)

2036

In [54]:
pd.read_sql_query("SELECT * FROM stockData", engine, parse_dates="Date").tail(3)

,Date,A,AAP,AAPL,ABB,ABM,ABR,ABT,ACGL,ACIW,...,PSA,PSTG,PSX,PVH,PWR,PXD,QDEL,QSR,QTWO,RAD
2033,2023-02-01,155.45,156.84,145.43,35.33,47.23,15.16,111.82,64.18,28.20,...,305.94,29.55,97.99,93.09,152.87,223.96,88.51,67.05,33.76,3.69
2034,2023-02-02,155.50,155.24,150.82,33.93,47.94,15.34,112.25,61.35,28.44,...,311.46,30.27,98.98,92.35,151.68,220.00,91.46,67.55,35.78,3.82
2035,2023-02-03,154.55,152.17,154.50,34.39,48.01,15.31,111.79,61.26,28.16,...,308.47,29.91,98.81,92.06,152.43,220.00,89.70,67.65,34.39,3.76


In [60]:
pd.read_sql_query("SELECT Date, RBA FROM stockData", engine, parse_dates="Date").tail(3)

OperationalError: (sqlite3.OperationalError) no such column: RBA
[SQL: SELECT Date, RBA FROM stockData]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [40]:
pd.read_sql("SELECT Date, COUNT(*) AS JustCounting FROM stockData WHERE A<35", engine, parse_dates="Date")

,Date,JustCounting
0,2015-08-21,58


In [41]:

# AVG
# COUNT
# DISTINCT
# LOWER
# MAX
# MIN
# UPPER
# WHERE
# LIKE
# ORDER BY
# LIMIT
# GROUP BY
# DESC

# UPDATE table SET column = value WHERE condition
# DELETE FROM table WHERE condition

# Data types: BLOB (Binary large object), INTEGER, NUMERIC (special number based info like dates, times etc...), REAL (foat number), TEXT

# NOT NULL is used to make sure that value was represented (text or number)

# SELECT column FROM table WHERE id IN (SELECT id FROM table WHERE condition = value) . First executes in brackets, then combines information

# SELECT column FROM table WHERE id IN (SELECT short_id FROM shorts WHERE short_id = (SELECT id FROM longs WHERE long = name)) ORDER BY column

# SELECT * FROM shows JOIN genres ON shows.id = genres.show_id WHERE title = 'the office'
# SELECT * FROM people WHERE name LIKE 'Steve C%'  % sign tells to choose anything after C

# CREATE INDEX column_index ON shows (column_name) . Makes searches much faster with seperate column index. B-trees

# variable_name = input("ticker: ")
# db.execute("SELECT * FROM table WHERE ticker = ?", variable_name)  dynamic routing

# race condition - many users do the same requests.

# to make race condition less problamatic there are few solutions: BEGIN TRANSACTION, COMMIT, ROLLBACK.
# db.execute("BEGIN TRANSACTION")
# rows = db.execute("SELECT * FROM data WHERE id = ?", id)
# likes = rows[0]["likes"]
# db.execute("UPDATE posts SET likes = ? WHERE id = ?", likes+1, id)
# db.execute("COMMIT")

#  '-- in SQL means ignore after dashes (comments out left code). Single qoute helps if DB code line is formatted, not used ?.

In [42]:
pd.read_sql("SELECT Date, A FROM stockData GROUP BY A", engine, parse_dates="Date").head(3)

,Date,A
0,2015-09-28,31.42
1,2015-09-29,31.76
2,2015-08-25,32.25
